<h2><center>
    Loading In Data
    </center></h2>

In [1]:
!pip install PyGithub --quiet

In [2]:
from github import Github
import os
import pickle
import base64
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
from ipywidgets import interact
import datetime as dt
sns.set()

In [3]:
with open("github.txt") as myfile:
    firstNlines=myfile.readlines()[0:2]
myfile.close()
g = Github(firstNlines[0].strip(), firstNlines[1])

In [4]:
repo=g.get_repo('CSSEGISandData/COVID-19')
contents = repo.get_contents("")

In [5]:
def get_sha_for_tag(repository, tag):
    """
    Returns a commit PyGithub object for the specified repository and tag.
    """
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha

    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError('No Tag or Branch exists with that name')
    return matched_tags[0].commit.sha

In [6]:
def download_directory(repository, sha, server_path, local_path='data_csse/'):
    """
    Download all contents at server_path with commit tag sha in
    the repository.
    """
    contents = repository.get_contents(server_path, ref=sha)
    if not os.path.exists(local_path):
        os.makedirs(local_path)
    for content in contents:
        if content.type == 'dir':
            download_directory(repository, sha, content.path)
        else:
            try:
                path = content.path
                file_content = repository.get_contents(path, ref=sha)
                file_data = base64.b64decode(file_content.content).decode('ascii')
                file_out = open(local_path+content.name, "w")
                file_out.write(local_path+file_data)
                file_out.close()
            except:
                pass

In [7]:
sha = get_sha_for_tag(repo, 'master')

In [8]:
download_directory(repo, sha, 'csse_covid_19_data/csse_covid_19_time_series/')

In [9]:
df = pd.read_csv('data_csse/02-01-2020.csv')

In [10]:
from os import listdir
from os.path import isfile, join
mypath = 'data_csse/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [11]:
def is_non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [12]:
timeseries_files = []
for file in onlyfiles:
    if is_non_zero_file(mypath + file) and file[:14]=='time_series_19':
        timeseries_files.append(mypath + file)

In [13]:
def preprocess_df(df, name):
    df.drop(columns=['data_csse/Province/State', 'Lat', 'Long'], inplace=True)
    df = df.groupby(['Country/Region']).agg('sum')
    df = df.transpose().reset_index()
    country_list = list(df.columns)[1:]
    df = pd.melt(df, id_vars='index', value_vars=country_list)
    df = df.rename(columns={'index':'Date', 'value':name})
    return df

In [14]:
confirmed_df = pd.read_csv('data_csse/time_series_19-covid-Confirmed.csv')
deaths_df = pd.read_csv('data_csse/time_series_19-covid-Deaths.csv')
recovered_df = pd.read_csv('data_csse/time_series_19-covid-Recovered.csv')

In [15]:
confirmed_df = preprocess_df(confirmed_df, 'Confirmed')
deaths_df = preprocess_df(deaths_df, 'Deaths')
recovered_df = preprocess_df(recovered_df, 'Recovered')

<h2><center>
    Preprocessing Data
    </center></h2>

In [16]:
confirmed_and_deaths = pd.merge(confirmed_df, deaths_df, how='inner', on=['Date', 'Country/Region'])

In [17]:
grouped_df = pd.merge(confirmed_and_deaths, recovered_df, how='inner', on=['Date', 'Country/Region'])

In [18]:
grouped_df['Active'] = grouped_df['Confirmed']-grouped_df['Deaths']-grouped_df['Recovered']

In [19]:
grouped_df['Datetime'] = grouped_df['Date'].apply(lambda x: pd.to_datetime(x))

In [20]:
pop_df = pd.read_csv('pop_df4.csv')

In [21]:
pop_df['Country/Region'] = pop_df['Country/Region'].apply(lambda x: x.replace('Mainland China', 'China'))

In [22]:
pop_df.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Mainland China', 'China')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Hong Kong SAR', 'Hong Kong')
grouped_df['Country/Region'] = grouped_df['Country/Region'].replace(['Korea, South', 'Republic of Korea'], 'South Korea')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('United Kingdom', 'UK')
grouped_df['Country/Region'] = grouped_df['Country/Region'].replace(['Taiwan*', 'Taipei and environs'], 'Taiwan')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Iran (Islamic Republic of)', 'Iran')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Viet Nam', 'Vietnam')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Macao SAR', 'Macau')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Republic of Ireland', 'Ireland')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Czechia', 'Czech Republic')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('occupied Palestinian territory', 'Palestine')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Russian Federation', 'Russia')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace(' Azerbaijan', 'Azerbaijan')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Holy See', 'Vatican City')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Republic of Moldova', 'Moldova')
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('Saint Martin', 'St. Martin')

In [24]:
grouped_df = pd.merge(grouped_df, pop_df, on='Country/Region')

In [25]:
grouped_df['Confirmed Cases Per 1M'] = grouped_df['Confirmed']*1000/grouped_df['PopTotal']

In [26]:
grouped_df['Country/Region'] = grouped_df['Country/Region'].str.replace('US', 'United States')

<h2><center>
    Data Visualization
    </center></h2>

In [28]:
@interact(Country = grouped_df['Country/Region'].sort_values().unique(), Confirmed=True, Active=True, Recovered=True, Deaths=True)
def viz(Country, Confirmed, Active, Recovered, Deaths):
    fig = px.line(title='Cases Over Time')
    if Confirmed:
        country_df = pd.DataFrame(grouped_df.loc[grouped_df['Country/Region']==Country].groupby('Datetime')['Confirmed'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Confirmed'], mode='lines', name='Confirmed Cases')
    if Active:
        country_df = pd.DataFrame(grouped_df.loc[grouped_df['Country/Region']==Country].groupby('Datetime')['Active'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Active'], mode='lines', name='Active Cases')
    if Recovered:
        country_df = pd.DataFrame(grouped_df.loc[grouped_df['Country/Region']==Country].groupby('Datetime')['Recovered'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Recovered'], mode='lines', name='Recovered Cases')
    if Deaths:
        country_df = pd.DataFrame(grouped_df.loc[grouped_df['Country/Region']==Country].groupby('Datetime')['Deaths'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Deaths'], mode='lines', name='Deceased Cases')
    fig.update_xaxes(title='Date', range=[dt.date(2020, 1, 22), dt.datetime.now()])
    fig.show()

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

In [29]:
fig = px.choropleth(grouped_df,
                        locations='Country/Region',
                        locationmode='country names',
                        color='Confirmed',
                        color_continuous_scale=['green', 'yellow',
                                                'orange', 'orangered', 'red'],
                        projection='natural earth',
                        animation_frame='Date',
                        range_color=[0, grouped_df['Confirmed'].max()])
fig.show()

In [30]:
pickle_out = open('COVID_Hopkins_df.pickle', 'wb')
pickle.dump(grouped_df, pickle_out)
pickle_out.close()